In [1]:
import numpy as np 
import matplotlib.pyplot as plt 
import sklearn.neighbors
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

import pytesseract
from PIL import Image
import cv2
import pandas as pd 
import skimage
from pathlib import Path

%matplotlib tk

## Dataset paths and loading

In [2]:
from common import *

c:\Users\admpdi\Code\personal\CMP197\pid_detect\notebook\common.py:257: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if len(img.shape) is 3 and not isinstance(padColor, (list, tuple, np.ndarray)): # color image but only one color provided


## Load sample image

In [3]:
image_id = 2
im, data = load_sample(image_id)

In [4]:
# Linetypes
def draw_pipelines(image, data=data):
    draw = image.copy()
    solid_lines = np.stack(data["lines"].query("type=='solid'")["box"])
    dashed_lines = np.stack(data["lines"].query("type=='dashed'")["box"])

    draw = cv2.drawContours(draw, solid_lines.reshape(-1,2,2), -1, (255, 255, 0), thickness=2)
    draw = cv2.drawContours(draw, dashed_lines.reshape(-1,2,2), -1, (0, 255, 255), thickness=2)
    return draw

def draw_detections(image, rects, classes,color=(255,0,0)):
    draw_rects(image, rects, thickness=8, color=color)
    for r,c in zip(rects,classes):
        cv2.putText(image, str(c), r.flatten()[:2], cv2.FONT_HERSHEY_PLAIN, 4, (0,0,0))

def draw_symbols(image, data=data, color=None, thickness=2):
    draw = image.copy()
    for i, group in data["symbols"].groupby("class"):
        color_ = color or (np.random.rand(3)*255).astype(np.uint8)
        symbols = np.stack(group["box"])
        draw_rects(draw, symbols, color=[int(c) for c in color_], thickness=thickness)
    return draw

def draw_gt_symbols(image, data=data,color=None, thickness=2):
    symbol_boxes = np.stack(data["symbols"]["box"])
    symbol_classes = np.stack(data["symbols"]["class"]).astype(int)
    draw_detections(image, symbol_boxes, symbol_classes, color=color)

def draw_text_boxes(image, data=data, color=(255,0,255), thickness=1):
    draw = image.copy()
    text_boxes = np.stack(data["words"]["box"])
    draw_rects(draw, text_boxes, color=color, thickness=thickness)
    return draw


In [18]:
cleanup = im.copy()

cleanup = draw_symbols(cleanup, color=(255,255,255), thickness=-1)
cleanup = draw_text_boxes(cleanup, color=(255,255,255), thickness=-1)

plt.imshow(cleanup)

gray = cv2.cvtColor(cleanup, cv2.COLOR_BGR2GRAY)
t, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)
plt.figure()
plt.imshow(thresh)

In [6]:
# skel = skimage.morphology.skeletonize(thresh//255, method="lee")
# plt.imshow(skel)

### HoughP

In [44]:
fig, axs = plt.subplots(1,2,sharex=True, sharey=True)
region = (slice(1000,-1000), slice(1000,-1000))

inp = cleanup

draw = np.zeros_like(inp[region].copy())

gray = cv2.cvtColor(inp[region], cv2.COLOR_BGR2GRAY)
t, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)
crop = thresh.copy()

skel = skimage.morphology.skeletonize(crop//255, method="lee")

linesP = cv2.HoughLinesP(crop, rho=1, theta=np.pi / 2, threshold=100, lines=None, minLineLength=20, maxLineGap=5)

if linesP is not None:
    for i in range(0, len(linesP)):
        l = linesP[i][0]
        
        color = np.random.randint(0,255,3)
        cv2.line(draw, (l[0], l[1]), (l[2], l[3]), tuple(int(c) for c in color), 3, cv2.LINE_AA)

    axs[0].imshow(draw)
    axs[1].imshow(crop)
    

In [15]:
plt.imshow(crop)